# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение данных

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score 

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Данные обрабатывали ранее - с ними все в порядке. В датасете 3214 объектов, из них: тариф смарт(0) - 2229, тариф ультра(1) - 985.

## Разбиение на выборки

Перед нами стоит задача бинарной классификации, целевой признак - 'is_ultra', остальные признаки помогут предсказать значения этого столбца.

Разобьем данные на три выборки: обучающую, валидационную и тестовую, в пропорции 3:1:1. Затем признаки каждой из выборок запишем в отдельные переменные.

In [5]:
data_train, data_other = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_other, test_size=0.5, random_state=12345)
print('Размер обучающей выборки:', len(data_train))
print('Размер валидационной выборки:', len(data_valid))
print('Размер тестовой выборки:', len(data_test))

Размер обучающей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


In [6]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

# проверим размер на обучающей выборке 
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


## Исследование моделей

Обучим каждую из моделей на обучающей выборке, переберем гиперпараметры и выберем модель с наилучшим результом accuracy на валидационной выборке.

**Дерево решений**

In [7]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train,target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid,predictions) 
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print("Accuracy лучшей модели:", best_result)
print("Глубина лучшей модели:", best_depth)

Accuracy лучшей модели: 0.7853810264385692
Глубина лучшей модели: 3


**Случайный лес**

In [8]:
best_model_forest = None
best_result = 0
best_depth = 0
best_est = 0
for est in range(1, 50):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model_forest = model
            best_result = result
            best_depth = depth
            best_est = est

print("Accuracy лучшей модели:", best_result)
print("Глубина лучшей модели:", best_depth)
print("Количество деревьев лучшей модели:", best_est)

Accuracy лучшей модели: 0.8087091757387247
Глубина лучшей модели: 8
Количество деревьев лучшей модели: 40


**Логистическая регрессия**

In [9]:
model = LogisticRegression(random_state=12345, solver='lbfgs') 
model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
result = accuracy_score(target_valid,predictions) 

print("Accuracy модели:", result)

Accuracy модели: 0.7107309486780715


Наилучшая модель - случайный лес (accuracy - 0,80) с 40 деревьями и глубиной, равной 8. Дальше работать будем с ней.

## Проверка модели на тестовой выборке

Проверим точность предсказаний модели случайного леса на тестовой выборке.

In [10]:
predictions = best_model_forest.predict(features_test) 
result = accuracy_score(target_test,predictions) 

print("Accuracy модели:", result)

Accuracy модели: 0.7962674961119751


Accuracy модели на тестовой выборке снизилась совсем незначительно, значит модель хорошо обучилась и не сильно промахивается в своих предсказаниях.

## Проверка модели на адекватность

Сравним точность модели случайного леса на тестовой выборке с предсказаниями DummyClassifier - простейшей модели, которая всегда предсказывает наиболее часто встречающийся класс (в данном случае - тариф смарт(0)). 

In [11]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_clf.fit(features_train, target_train)
predictions = dummy_clf.predict(features_test)
result = dummy_clf.score(predictions, target_test)

print("Accuracy модели:", result)

Accuracy модели: 0.6842923794712286


Accuracy модели случайного леса на 12% выше, чем DummyClassifier, значит модель адекватна и с ней можно работать.

**Вывод**

Для подбора наиболее подходящего тарифа на основе данных о поведении клиентов были обучены 3 модели: дерево решений, случайный лес и логистическая регрессия.

Наилучший результат показала модель случайного леса с accuracy = 0,80 и следующими гиперпараметрами:
- max_depth = 8
- n_estimators = 40

Модель дерева решений также достаточно точна (0,78). Наименьшая точность у логистической регрессии (0,71).

При проверке модели случайного леса на тестовой выборке точность снизилась совсем не существенно, а при сравнении с простейшей моделью DummyClassifier точность оказалась более, чем на 10% выше.

Исходя из оценки точности и проверки на адекватность наиболее подходящяя модель для решения задачи классификации - модель случайного леса.